<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllCF_Norm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/InfluenceAnalysis/crowdFlower/crowdFlower_clean.csv')
df = df.sample(frac=1,random_state=32)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



In [4]:
df.head()

,Unnamed: 0,emotion,tweets
35975,35975,6,katofawesome praying for love in a lap dance a...
17590,17590,1,trying to fix my internet connectionguess my p...
38071,38071,0,starting an account here on twitter
23304,23304,1,dipfico hmm wrong link ignore my tweet
26990,26990,0,mzunyque thanks before the major chop


In [5]:
category_dict = {0:[1,0,0,0,0,0,0,0,0,0,0,0,0],1:[0,1,0,0,0,0,0,0,0,0,0,0,0],2:[0,0,1,0,0,0,0,0,0,0,0,0,0],3:[0,0,0,1,0,0,0,0,0,0,0,0,0],4:[0,0,0,0,1,0,0,0,0,0,0,0,0],5:[0,0,0,0,0,1,0,0,0,0,0,0,0],6:[0,0,0,0,0,0,1,0,0,0,0,0,0],7:[0,0,0,0,0,0,0,1,0,0,0,0,0],8:[0,0,0,0,0,0,0,0,1,0,0,0,0],9:[0,0,0,0,0,0,0,0,0,1,0,0,0],10:[0,0,0,0,0,0,0,0,0,0,1,0,0],11:[0,0,0,0,0,0,0,0,0,0,0,1,0],12:[0,0,0,0,0,0,0,0,0,0,0,0,1]}
data_tweet = [x.lower().split() for x in df['tweets']]
data_cat = np.array([category_dict[x] for x in df['emotion']])

In [6]:
print(data_tweet[:5])
print(data_cat[:5])

[['katofawesome', 'praying', 'for', 'love', 'in', 'a', 'lap', 'dance', 'and', 'paying', 'in', 'naivety', 'lt--one', 'of', 'my', 'fav', 'songs'], ['trying', 'to', 'fix', 'my', 'internet', 'connectionguess', 'my', 'prayers', 'have', 'been', 'answered', 'and', 'i', 'wont', 'have', 'any', 'study', 'distractions', 'ugh'], ['starting', 'an', 'account', 'here', 'on', 'twitter'], ['dipfico', 'hmm', 'wrong', 'link', 'ignore', 'my', 'tweet'], ['mzunyque', 'thanks', 'before', 'the', 'major', 'chop']]
[[0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]]


In [7]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [8]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [9]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [10]:
batch_size = 100
Max_input_size = max([len(x) for x in data_tweet])
print(Max_input_size)

35


In [11]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([w2v.wv.get_vector(i) for i in x if i in vocab]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([vocab[i] for i in x if i in vocab.keys()]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(data_tweet, data_cat, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
result_table = [0,0,0,0,0,0,0,0,0]

In [13]:
epochs = 40

In [14]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
sg_train_gen=vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen(w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_sg_lstm = model_sg.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


Epoch 1/40
257/257 [==============================] - 90s 342ms/step - loss: 2.2613 - accuracy: 0.2098 - val_loss: 2.1395 - val_accuracy: 0.2177
Epoch 2/40
257/257 [==============================] - 81s 317ms/step - loss: 2.1434 - accuracy: 0.2286 - val_loss: 2.1108 - val_accuracy: 0.2623
Epoch 3/40
257/257 [==============================] - 81s 317ms/step - loss: 2.1135 - accuracy: 0.2533 - val_loss: 2.0807 - val_accuracy: 0.2652
Epoch 4/40
257/257 [==============================] - 79s 307ms/step - loss: 2.0783 - accuracy: 0.2715 - val_loss: 2.0664 - val_accuracy: 0.2789
Epoch 5/40
257/257 [==============================] - 80s 312ms/step - loss: 2.0481 - accuracy: 0.2934 - val_loss: 2.0527 - val_accuracy: 0.2823
Epoch 6/40
257/257 [==============================] - 79s 309ms/step - loss: 2.0302 - accuracy: 0.3046 - val_loss: 2.0378 - val_accuracy: 0.2915
Epoch 7/40
257/257 [==============================] - 81s 314ms/step - loss: 2.0134 - accuracy: 0.3110 - val_loss: 2.0315 - val_ac

In [15]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


Epoch 1/40
257/257 [==============================] - 92s 346ms/step - loss: 2.2401 - accuracy: 0.2129 - val_loss: 2.0768 - val_accuracy: 0.2677
Epoch 2/40
257/257 [==============================] - 90s 351ms/step - loss: 2.0679 - accuracy: 0.2814 - val_loss: 2.0376 - val_accuracy: 0.2929
Epoch 3/40
257/257 [==============================] - 90s 351ms/step - loss: 2.0251 - accuracy: 0.3010 - val_loss: 2.0291 - val_accuracy: 0.2980
Epoch 4/40
257/257 [==============================] - 90s 351ms/step - loss: 2.0086 - accuracy: 0.3065 - val_loss: 2.0255 - val_accuracy: 0.3034
Epoch 5/40
257/257 [==============================] - 92s 358ms/step - loss: 1.9899 - accuracy: 0.3128 - val_loss: 2.0223 - val_accuracy: 0.3009
Epoch 6/40
257/257 [==============================] - 89s 347ms/step - loss: 1.9737 - accuracy: 0.3189 - val_loss: 2.0188 - val_accuracy: 0.3038
Epoch 7/40
257/257 [==============================] - 89s 346ms/step - loss: 1.9585 - accuracy: 0.3259 - val_loss: 2.0126 - val_ac

In [16]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_rnn = model_sg_rnn.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)


result_table[2]=history[1]
print("SG_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


Epoch 1/40
257/257 [==============================] - 74s 285ms/step - loss: 2.2244 - accuracy: 0.2041 - val_loss: 2.1405 - val_accuracy: 0.2157
Epoch 2/40
257/257 [==============================] - 72s 282ms/step - loss: 2.1436 - accuracy: 0.2146 - val_loss: 2.1370 - val_accuracy: 0.2154
Epoch 3/40
257/257 [==============================] - 72s 281ms/step - loss: 2.1443 - accuracy: 0.2162 - val_loss: 2.1388 - val_accuracy: 0.2158
Epoch 4/40
257/257 [==============================] - 72s 281ms/step - loss: 2.1469 - accuracy: 0.2167 - val_loss: 2.1400 - val_accuracy: 0.2152
Epoch 5/40
257/257 [==============================] - 73s 285ms/step - loss: 2.1417 - accuracy: 0.2177 - val_loss: 2.1395 - val_accuracy: 0.2157
Epoch 6/40
257/257 [==============================] - 72s 281ms/step - loss: 2.1422 - accuracy: 0.2166 - val_loss: 2.1389 - val_accuracy: 0.2137
Epoch 7/40
257/257 [==============================] - 71s 277ms/step - loss: 2.1397 - accuracy: 0.2228 - val_loss: 2.1401 - val_ac

In [17]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_lstm = model_cbow.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


Epoch 1/40
257/257 [==============================] - 82s 312ms/step - loss: 2.2656 - accuracy: 0.2139 - val_loss: 2.1350 - val_accuracy: 0.2428
Epoch 2/40
257/257 [==============================] - 80s 314ms/step - loss: 2.1354 - accuracy: 0.2371 - val_loss: 2.0935 - val_accuracy: 0.2700
Epoch 3/40
257/257 [==============================] - 81s 318ms/step - loss: 2.0975 - accuracy: 0.2626 - val_loss: 2.0694 - val_accuracy: 0.2792
Epoch 4/40
257/257 [==============================] - 79s 309ms/step - loss: 2.0591 - accuracy: 0.2880 - val_loss: 2.0525 - val_accuracy: 0.2863
Epoch 5/40
257/257 [==============================] - 78s 303ms/step - loss: 2.0344 - accuracy: 0.3030 - val_loss: 2.0430 - val_accuracy: 0.2912
Epoch 6/40
257/257 [==============================] - 83s 322ms/step - loss: 2.0215 - accuracy: 0.3121 - val_loss: 2.0296 - val_accuracy: 0.2966
Epoch 7/40
257/257 [==============================] - 82s 320ms/step - loss: 2.0041 - accuracy: 0.3177 - val_loss: 2.0260 - val_ac

In [18]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


Epoch 1/40
257/257 [==============================] - 97s 364ms/step - loss: 2.2392 - accuracy: 0.2231 - val_loss: 2.0806 - val_accuracy: 0.2692
Epoch 2/40
257/257 [==============================] - 95s 369ms/step - loss: 2.0684 - accuracy: 0.2828 - val_loss: 2.0367 - val_accuracy: 0.3015
Epoch 3/40
257/257 [==============================] - 92s 358ms/step - loss: 2.0269 - accuracy: 0.3040 - val_loss: 2.0260 - val_accuracy: 0.3060
Epoch 4/40
257/257 [==============================] - 90s 351ms/step - loss: 2.0088 - accuracy: 0.3111 - val_loss: 2.0161 - val_accuracy: 0.3063
Epoch 5/40
257/257 [==============================] - 93s 364ms/step - loss: 1.9892 - accuracy: 0.3161 - val_loss: 2.0100 - val_accuracy: 0.3097
Epoch 6/40
257/257 [==============================] - 93s 363ms/step - loss: 1.9693 - accuracy: 0.3245 - val_loss: 2.0084 - val_accuracy: 0.3106
Epoch 7/40
257/257 [==============================] - 91s 356ms/step - loss: 1.9517 - accuracy: 0.3313 - val_loss: 2.0051 - val_ac

In [19]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_rnn = model_cbow_rnn.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


Epoch 1/40
257/257 [==============================] - 73s 284ms/step - loss: 2.2202 - accuracy: 0.1966 - val_loss: 2.1419 - val_accuracy: 0.2155
Epoch 2/40
257/257 [==============================] - 70s 275ms/step - loss: 2.1428 - accuracy: 0.2167 - val_loss: 2.1378 - val_accuracy: 0.2154
Epoch 3/40
257/257 [==============================] - 69s 271ms/step - loss: 2.1434 - accuracy: 0.2155 - val_loss: 2.1386 - val_accuracy: 0.2158
Epoch 4/40
257/257 [==============================] - 71s 277ms/step - loss: 2.1440 - accuracy: 0.2159 - val_loss: 2.1401 - val_accuracy: 0.2152
Epoch 5/40
257/257 [==============================] - 72s 280ms/step - loss: 2.1409 - accuracy: 0.2148 - val_loss: 2.1397 - val_accuracy: 0.2148
Epoch 6/40
257/257 [==============================] - 72s 280ms/step - loss: 2.1409 - accuracy: 0.2160 - val_loss: 2.1393 - val_accuracy: 0.2142
Epoch 7/40
257/257 [==============================] - 75s 293ms/step - loss: 2.1379 - accuracy: 0.2229 - val_loss: 2.1408 - val_ac

In [20]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
glove_train_gen=glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_lstm = model_glove.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


Epoch 1/40
257/257 [==============================] - 18s 64ms/step - loss: 2.2249 - accuracy: 0.2304 - val_loss: 1.9880 - val_accuracy: 0.3048
Epoch 2/40
257/257 [==============================] - 16s 61ms/step - loss: 1.9736 - accuracy: 0.3101 - val_loss: 1.9076 - val_accuracy: 0.3391
Epoch 3/40
257/257 [==============================] - 16s 61ms/step - loss: 1.9045 - accuracy: 0.3468 - val_loss: 1.8759 - val_accuracy: 0.3562
Epoch 4/40
257/257 [==============================] - 16s 61ms/step - loss: 1.8622 - accuracy: 0.3677 - val_loss: 1.8642 - val_accuracy: 0.3577
Epoch 5/40
257/257 [==============================] - 16s 61ms/step - loss: 1.8389 - accuracy: 0.3735 - val_loss: 1.8590 - val_accuracy: 0.3595
Epoch 6/40
257/257 [==============================] - 16s 61ms/step - loss: 1.8213 - accuracy: 0.3799 - val_loss: 1.8488 - val_accuracy: 0.3622
Epoch 7/40
257/257 [==============================] - 16s 61ms/step - loss: 1.7951 - accuracy: 0.3897 - val_loss: 1.8466 - val_accuracy:

In [21]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


Epoch 1/40
257/257 [==============================] - 30s 105ms/step - loss: 2.1450 - accuracy: 0.2606 - val_loss: 1.8899 - val_accuracy: 0.3451
Epoch 2/40
257/257 [==============================] - 26s 103ms/step - loss: 1.8755 - accuracy: 0.3503 - val_loss: 1.8478 - val_accuracy: 0.3649
Epoch 3/40
257/257 [==============================] - 26s 103ms/step - loss: 1.8158 - accuracy: 0.3724 - val_loss: 1.8386 - val_accuracy: 0.3645
Epoch 4/40
257/257 [==============================] - 27s 103ms/step - loss: 1.7812 - accuracy: 0.3863 - val_loss: 1.8334 - val_accuracy: 0.3663
Epoch 5/40
257/257 [==============================] - 27s 103ms/step - loss: 1.7532 - accuracy: 0.3996 - val_loss: 1.8348 - val_accuracy: 0.3645
Epoch 6/40
257/257 [==============================] - 27s 104ms/step - loss: 1.7224 - accuracy: 0.4115 - val_loss: 1.8435 - val_accuracy: 0.3629
Epoch 7/40
257/257 [==============================] - 27s 105ms/step - loss: 1.6941 - accuracy: 0.4225 - val_loss: 1.8594 - val_ac

In [22]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_rnn = model_glove_rnn.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


Epoch 1/40
257/257 [==============================] - 7s 26ms/step - loss: 2.2792 - accuracy: 0.1959 - val_loss: 2.1215 - val_accuracy: 0.2643
Epoch 2/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1137 - accuracy: 0.2542 - val_loss: 2.1087 - val_accuracy: 0.2129
Epoch 3/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1271 - accuracy: 0.2255 - val_loss: 2.1352 - val_accuracy: 0.2157
Epoch 4/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1430 - accuracy: 0.2164 - val_loss: 2.1322 - val_accuracy: 0.2149
Epoch 5/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1140 - accuracy: 0.2391 - val_loss: 2.1326 - val_accuracy: 0.2215
Epoch 6/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1411 - accuracy: 0.2178 - val_loss: 2.1324 - val_accuracy: 0.2202
Epoch 7/40
257/257 [==============================] - 6s 25ms/step - loss: 2.1203 - accuracy: 0.2281 - val_loss: 2.0987 - val_accuracy: 0.2368

In [23]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])
# print(history_sg_bi.history)
# plt.title("Skip-Gram + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_sg_bi.history['loss'],c='b',label='loss')
# plt.plot(history_sg_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()
# plt.title("GloVe + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_glove_bi.history['loss'],c='b',label='loss')
# plt.plot(history_glove_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()

	Lstm			BiLstm			Rnn
sg [0.3163749873638153, 0.29074999690055847, 0.23774999380111694]
cbow [0.30937498807907104, 0.2879999876022339, 0.22624999284744263]
glove [0.31200000643730164, 0.28587499260902405, 0.2574999928474426]
